### Import libraries

In [1]:
from aif360.datasets import BankDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import LFR
from aif360.algorithms.inprocessing import PrejudiceRemover

2023-01-20 15:06:30.061286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load the bankdata
protected_attribute_maps = [{1.0: 'married', 0.0: 'unmarried'}]
dataset_orig = BankDataset(
            protected_attribute_names=['marital'],          
            privileged_classes=[['married']], 
            features_to_drop=['campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
            categorical_features=['job', 'education', 'default',
                    'housing', 'loan', 'contact', 'month', 'day_of_week'],
            metadata={'protected_attribute_maps': protected_attribute_maps}
        )
privileged_groups = [{'marital': 1}]
unprivileged_groups = [{'marital': 0}]

* Split the dataset with ratio into training data and test data
* Measure the fairness metrics of the training and test data

In [3]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=None)

    
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)

# print the metric values 
print('SPD', round(metric_orig_train.mean_difference(), 2))
print('DI', round(metric_orig_train.disparate_impact(), 2))


SPD 0.18
DI 1.7


In [4]:
# train the dataset with Logistic Regression model
def Logistic_Regression(training_data, test_data):
    model = LogisticRegression(random_state=0, max_iter = 1000)
    # train model
    model.fit(training_data.features, training_data.labels.ravel())
    # test the model
    prediction_label = model.predict(test_data.features)
    prediction = dataset_orig_test.copy()
    prediction.labels = prediction_label
    # return the prediction on the test data
    return prediction

prediction = Logistic_Regression(dataset_orig_train, dataset_orig_test)

In [5]:
# measure the accuracy and the fairness metrics on the prediction
def get_prediction_metrics(prediction):
    metric = ClassificationMetric(
                        dataset_orig_test, prediction,
                        unprivileged_groups=unprivileged_groups,
                        privileged_groups=privileged_groups)

    accuracy = metric.accuracy()
    print('accuracy', accuracy)
    print(round(metric.statistical_parity_difference(), 2))
    print(round(metric.disparate_impact(), 2))
    print(round(metric.equal_opportunity_difference(), 2))
    print(round(metric.average_odds_difference(), 2))

get_prediction_metrics(prediction)

accuracy 0.7973333333333333
0.16
1.7
0.08
0.05


In [9]:
# Pre-processing: reweighing method
RW_model = Reweighing(unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)
dataset_RW_train = RW_model.fit_transform(dataset_orig_train)
metric_RW_train = BinaryLabelDatasetMetric(dataset_RW_train, 
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)

# print the metric values 
print('SPD', round(metric_RW_train.mean_difference(), 2))
print('DI', round(metric_RW_train.disparate_impact(), 2))

SPD 0.0
DI 1.0


In [10]:
# Pre-processing: Learning fair representations
LFR_model = LFR(unprivileged_groups=unprivileged_groups, 
    privileged_groups=privileged_groups,
    verbose=0, seed=10)
LFR_model = LFR_model.fit(dataset_orig_train)
dataset_LFR_train = LFR_model.transform(dataset_orig_train)

metric_LFR_train = BinaryLabelDatasetMetric(dataset_LFR_train, 
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
print('SPD', round(metric_RW_train.mean_difference(), 2))
print('DI', round(metric_RW_train.disparate_impact(), 2))

SPD 0.0
DI 1.0


In [11]:
# training the original model with the processed training data
prediction = Logistic_Regression(dataset_LFR_train, dataset_orig_test)
get_prediction_metrics(prediction)

prediction = Logistic_Regression(dataset_RW_train, dataset_orig_test)
get_prediction_metrics(prediction)

accuracy 0.7013333333333334
0.0
1.01
-0.11
-0.09
accuracy 0.7973333333333333
0.16
1.7
0.08
0.05


In [12]:
# in-processing: Prejudice remover
model = PrejudiceRemover(eta=0.1)
model.fit(dataset_orig_train)
prediction = model.predict(dataset_orig_test)
get_prediction_metrics(prediction)

accuracy 0.798
0.17
1.84
0.13
0.08


In [14]:
# in-processing: Adversarial debiasing
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

tf.reset_default_graph()
sess = tf.Session()
num_epochs = 50
classifier_num_hidden_units = 200
model = AdversarialDebiasing(privileged_groups = privileged_groups,
                            unprivileged_groups = unprivileged_groups,
                            scope_name='debiased_classifier',
                            debias=True,
                            sess=sess)
model.fit(dataset_RW_train)
prediction = model.predict(dataset_orig_test)
get_prediction_metrics(prediction)

epoch 0; iter: 0; batch classifier loss: 7.415534; batch adversarial loss: 0.698311
epoch 1; iter: 0; batch classifier loss: 4.718514; batch adversarial loss: 0.696130
epoch 2; iter: 0; batch classifier loss: 3.182117; batch adversarial loss: 0.691449
epoch 3; iter: 0; batch classifier loss: 3.366002; batch adversarial loss: 0.663497
epoch 4; iter: 0; batch classifier loss: 3.189120; batch adversarial loss: 0.676688
epoch 5; iter: 0; batch classifier loss: 2.056461; batch adversarial loss: 0.695969
epoch 6; iter: 0; batch classifier loss: 1.870984; batch adversarial loss: 0.695920
epoch 7; iter: 0; batch classifier loss: 1.150717; batch adversarial loss: 0.678812
epoch 8; iter: 0; batch classifier loss: 1.191106; batch adversarial loss: 0.667851
epoch 9; iter: 0; batch classifier loss: 1.233681; batch adversarial loss: 0.680741
epoch 10; iter: 0; batch classifier loss: 0.692564; batch adversarial loss: 0.669964
epoch 11; iter: 0; batch classifier loss: 0.601915; batch adversarial loss: